In [1]:
from model import *
from data import *
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint 
import segmentation_models as sm



Segmentation Models: using `tf.keras` framework.


In [18]:
BACKBONE = 'resnet34' #Pretrained backbone


Train form data in dataset using data augmentation

In [2]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(3,'dataset','images','labels',data_gen_args,save_to_dir = None)
model = sm.Unet(BACKBONE)
model.compile('Adam',loss=sm.losses.bce_jaccard_loss,metrics=['acc'])
model_checkpoint = ModelCheckpoint('pretrained_unet.hdf5', monitor='acc',verbose=1, save_best_only=True)
model.fit_generator(train_generator,steps_per_epoch=800,epochs=5,validation_data=val_generator,validation_steps=120, callbacks=[model_checkpoint])

85524480/85521592 [==============================] - 4s 0us/step


NameError: name 'train_generator' is not defined

Test model

In [ ]:
model=satellite_unet(pretrained='sat_unet.hdf5')
X_test,names=test("dataset/unlabelled")
preds=model.predict(X_test)
preds=preds>0.5
results=predToImgs(preds)
saveResults("dataset/preds",results,names)

In [3]:
def test(test_path,target_size=(256,256), color_mode = "rgb"):
    X_test=[]
    names=[]
    for filename in os.listdir(test_path):
        name, ext = os.path.splitext(filename)
        if ext!=".png" and ext!=".jpg":
            continue
        names.append(filename)
        img=load_img(os.path.join(test_path,filename),target_size=(256,256), color_mode = color_mode)
        img=img_to_array(img)/255
        X_test.append(img.copy())
    X_test_label = np.array(X_test)
    return X_test_label

test_img = test('dataset/images_satelite')
# test_img = test_img[1:,:,:,:]
print(test_img.shape)

test_label = test('dataset/labels_satelite', color_mode = "grayscale")
print(test_label.shape)


(14, 256, 256, 3)
(14, 256, 256, 1)


In [46]:
model=satellite_unet(pretrained='sat_unet.hdf5')

score, acc = model.evaluate(test_img, test_label)
print('Test score:', score)
print('Test accuracy:', acc)

18/18 [==============================] - 1s 52ms/sample - loss: 1.5818 - accuracy: 0.6035
Test score: 1.581809163093567
Test accuracy: 0.6035216


In [49]:
data_gen_args = dict(rotation_range=0.2,
                    width_shift_range=0.05,
                    height_shift_range=0.05,
                    shear_range=0.05,
                    zoom_range=0.05,
                    horizontal_flip=True,
                    vertical_flip=True,
                    fill_mode='nearest')
myGene = trainGenerator(3,'dataset','images','labels',data_gen_args,save_to_dir = None)
model = sm.Unet(BACKBONE)
model.compile(tf.keras.optimizers.Adam(1e-4),loss=sm.losses.bce_jaccard_loss,metrics=['acc'])
model_checkpoint = ModelCheckpoint('pretrained_unet.hdf5', monitor='acc',verbose=1, save_best_only=True)
model.fit_generator(myGene,steps_per_epoch=2000,epochs=5,callbacks=[model_checkpoint])

In [15]:
model=satellite_unet(pretrained='sat_unet_IoU.hdf5')

score, acc = model.evaluate(test_img, test_label)
print('Test score:', score)
print('Test accuracy:', acc)

18/18 [==============================] - 1s 60ms/sample - loss: 1.9629 - mean_io_u_1: 0.4234
Test score: 1.9629113674163818
Test accuracy: 0.4234361


In [4]:
# Tried data from the same distribution, split data from our satelite images into train and validation sets

def trainGenerator(batch_size,train_path,image_folder,mask_folder,aug_dict=data_gen_args,image_color_mode = "rgb",
                    mask_color_mode = "grayscale",image_save_prefix  = "image",mask_save_prefix  = "mask",
                    flag_multi_class = False,num_class = 2,save_to_dir = None,
                   target_size = (256,256),seed = 1):
    '''
    can generate image and mask at the same time
    use the same seed for image_datagen and mask_datagen to ensure the transformation for image and mask is the same
    if you want to visualize the results of generator, set save_to_dir = "your path"
    '''
    image_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    mask_datagen = ImageDataGenerator(**aug_dict, validation_split=0.2, rescale=1./255)
    
    image_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'training',
        seed = seed) 
    image_val_generator = image_datagen.flow_from_directory(
        train_path,
        classes = [image_folder],
        class_mode = None,
        color_mode = image_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = image_save_prefix,
        subset = 'validation',
        seed = seed)
    
    mask_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'training',
        seed = seed)
    mask_val_generator = mask_datagen.flow_from_directory(
        train_path,
        classes = [mask_folder],
        class_mode = None,
        color_mode = mask_color_mode,
        target_size = target_size,
        batch_size = batch_size,
        save_to_dir = save_to_dir,
        save_prefix  = mask_save_prefix,
        subset = 'validation',
        seed = seed)
    
        
    return image_generator, image_val_generator, mask_generator, mask_val_generator


In [5]:
image_generator, image_val_generator, mask_generator, mask_val_generator = trainGenerator(8,'dataset','images_satelite','labels_satelite',data_gen_args)

Found 12 images belonging to 1 classes.
Found 2 images belonging to 1 classes.
Found 12 images belonging to 1 classes.
Found 2 images belonging to 1 classes.


In [6]:
def train(image_generator, mask_generator, flag_multi_class = False, num_class = 2):
    train_generator = zip(image_generator, mask_generator)
    for (img,mask) in train_generator:
        img,mask = adjustData(img,mask,flag_multi_class,num_class)       
        yield (img,mask)
        
def val(image_val_generator, mask_val_generator, flag_multi_class = False, num_class = 2):
    val_generator = zip(image_val_generator, mask_val_generator)
    for (img_val,mask_val) in val_generator:
        img_val,mask_val = adjustData(img_val,mask_val,flag_multi_class,num_class)       
        yield (img_val,mask_val)
        
train_generator = train(image_generator, mask_generator)
val_generator = val(image_val_generator, mask_val_generator)

In [19]:
model = sm.Unet(BACKBONE,encoder_weights='imagenet',input_shape=(256,256,3),encoder_freeze=True)
model.compile(tf.keras.optimizers.Adam(),loss=sm.losses.bce_jaccard_loss,metrics=['acc'])
model_checkpoint = ModelCheckpoint('pretrained_unet.hdf5', monitor='val_acc',verbose=1, save_best_only=True)
model.fit_generator(train_generator,steps_per_epoch=1000,epochs=3,validation_data=val_generator,validation_steps=100, callbacks=[model_checkpoint])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1000 steps, validate for 100 steps
Epoch 1/3
 999/1000 [============================>.] - ETA: 0s - loss: 0.2531 - acc: 0.9038
Epoch 00001: val_acc improved from -inf to 0.82981, saving model to pretrained_unet.hdf5
1000/1000 [==============================] - 187s 187ms/step - loss: 0.2529 - acc: 0.9038 - val_loss: 1.0387 - val_acc: 0.8298
Epoch 2/3
 999/1000 [============================>.] - ETA: 0s - loss: 0.0873 - acc: 0.9164
Epoch 00002: val_acc improved from 0.82981 to 0.84140, saving model to pretrained_unet.hdf5
1000/1000 [==============================] - 181s 181ms/step - loss: 0.0872 - acc: 0.9164 - val_loss: 1.1597 - val_acc: 0.8414
Epoch 3/3
 999/1000 [============================>.] - ETA: 0s - loss: 0.0662 - acc: 0.9169
Epoch 00003: val_acc improved from 0.84140 to 0.85904, saving model to pretrained_unet.hdf5
1000/1000 [==============================] - 180s 180ms/step - loss: 0.0662 - acc: 0.9169 - val_loss: 

In [22]:
for layer in model.layers:
    layer.trainable = True
model.compile(tf.keras.optimizers.Adam(1e-6),loss=sm.losses.bce_jaccard_loss,metrics=['acc'])
model.fit_generator(train_generator,steps_per_epoch=500,epochs=10,validation_data=val_generator,validation_steps=100, callbacks=[model_checkpoint])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 500 steps, validate for 100 steps
Epoch 1/10
499/500 [============================>.] - ETA: 0s - loss: 0.0428 - acc: 0.9173
Epoch 00001: val_acc did not improve from 0.87073
500/500 [==============================] - 99s 197ms/step - loss: 0.0427 - acc: 0.9173 - val_loss: 0.9836 - val_acc: 0.8654
Epoch 2/10
499/500 [============================>.] - ETA: 0s - loss: 0.0450 - acc: 0.9173
Epoch 00002: val_acc did not improve from 0.87073
500/500 [==============================] - 94s 188ms/step - loss: 0.0451 - acc: 0.9173 - val_loss: 1.0076 - val_acc: 0.8636
Epoch 3/10
499/500 [============================>.] - ETA: 0s - loss: 0.0534 - acc: 0.9165
Epoch 00003: val_acc did not improve from 0.87073
500/500 [==============================] - 94s 188ms/step - loss: 0.0534 - acc: 0.9165 - val_loss: 1.0208 - val_acc: 0.8616
Epoch 4/10
499/500 [============================>.] - ETA: 0s - loss: 0.0528 - acc: 0.9166
Epoch 00004: val_acc

In [23]:
test_img = test('dataset/test_images')
test_label = test('dataset/test_labels', color_mode = "grayscale")
#model=satellite_unet(pretrained='pretrained_unet.hdf5')

score= model.evaluate(test_img, test_label)
print('Test score:', score)
#print('Test accuracy:', acc)
# print('Test IoU:', Iou)

4/4 [==============================] - 1s 198ms/sample - loss: 1.9746 - acc: 0.8629
Test score: [1.9745510816574097, 0.86291885]


In [24]:
def test2(test_path,target_size=(256,256)):
    X_test=[]
    names=[]
    for filename in os.listdir(test_path):
        name, ext = os.path.splitext(filename)
        if ext!=".png" and ext!=".jpg":
            continue
        names.append(filename)
        img=load_img(os.path.join(test_path,filename),target_size=target_size)
        img=img_to_array(img)/255
        X_test.append(img.copy())
    return np.array(X_test),names
    

In [25]:
### TO DO ####
import os

#model=satellite_unet(pretrained='sat_unet_new.hdf5')
X_test,names=test2("dataset/test_images")
preds=model.predict(X_test)
preds=preds>0.5
results=predToImgs(preds)
saveResults(os.getcwd()+"/dataset/generator_images",results,names,empty_dir=False)